In [46]:
import pandas as pd
import concurrent.futures
import requests
from tqdm import tqdm
import os
import feedparser

In [48]:
# https://www.data.gouv.fr/fr/datasets/r/9e19b1a5-4c30-46ca-a156-54c120a3c5fe
df = pd.read_csv("/home/nicolas/Downloads/depot-legal-du-web-liste-podcasts.csv")
df = df[['UGC_TECH_ID', 'UGC_TECH_DATE_ENTREE_COLLECTE', 'UGC_TECH_RSS_URL']].copy()


,UGC_TECH_ID,UGC_USER_NAME,UGC_USER_DESCRIPTION,UGC_DOC_DESCRIPTION,UGC_DOC_CATEGORIE,UGC_DOC_DESCRIPTEURS,UGC_DOC_GENRES,UGC_DOC_TYPE_PROGRAMME,UGC_DOC_CHAINES,UGC_DOC_EMISSIONS,UGC_TECH_DATE_ENTREE_COLLECTE,UGC_TECH_RSS_URL,UGC_TECH_RSS_URL_REDIRECT,UGC_TECH_IMAGE,UGC_TECH_LINK_URL,UGC_TECH_AUTEUR,UGC_TECH_TITRE
0,RMC__Le_grand_oral_des_GG,RMC - Le grand oral des GG,L'actualité du jour commentée à chaud avec imp...,à compléter,Talk Radio,radio,Web TV,Émission native web,NaN,NaN,2020-07-30,http://podcast.rmc.fr/channel49/RMCInfochannel...,http://podcast.rmc.fr/channel49/RMCInfochannel...,https://frontrmcimg.streamakaci.com/images/140...,http://rmc.bfmtv.com/mediaplayer/podcast/l-inv...,RMC,Le grand oral des GG
1,rfi-ensemble_a_la_maison,RFI-Ensemble à la maison,Les réseaux sociaux sont devenus essentiels po...,Emission mise en place durant le confinement l...,NaN,téléchargement (podcast)\nInternet\nsite inter...,NaN,NaN,NaN,NaN,2020-06-04,https://apis.rfi.fr/products/get_product/rfi_g...,NaN,https://s.rfi.fr/media/display/e7cc4546-7266-1...,http://www.rfi.fr/fr/podcasts/ensemble-%C3%A0-...,RFI,Ensemble à la maison
2,rfi-la_une_de_la_presse_cette_semaine_a_kinshasa,RFI-La une de la presse cette semaine à Kinshasa,"Toutes les semaines, RFI vous propose de feuil...",Chronique de Kamanda wa Kamanda Muzembe consac...,NaN,téléchargement (podcast)\nRépublique démocrati...,NaN,NaN,NaN,NaN,2020-06-04,https://apis.rfi.fr/products/get_product/rfi_g...,NaN,https://s.rfi.fr/media/display/613b548e-5fd0-1...,http://www.rfi.fr/fr/podcasts/une-presse-cette...,RFI,La une de la presse cette semaine à Kinshasa
3,rfi-le_regard_du_monde_afrique,RFI-Le regard du «Monde Afrique»,"Chaque samedi, RFI présente un article du jour...",Chronique hebdomadaire consacrée à l'actualité...,NaN,téléchargement (podcast)\nAfrique\nactualité\n...,NaN,NaN,NaN,NaN,2020-06-04,https://apis.rfi.fr/products/get_product/rfi_g...,NaN,https://s.rfi.fr/media/display/da830c0e-9eb6-1...,http://www.rfi.fr/fr/podcasts/le-regard-monde-...,RFI,Le regard du «Monde Afrique»
4,rfi-sessionlab,RFI-SessionLab,SessionLab est un programme musical diffusé un...,Emission musicale hebdomadaire uniquement diff...,NaN,téléchargement (podcast)\nmusique\nchanteur\nm...,NaN,NaN,NaN,NaN,2020-06-04,https://apis.rfi.fr/products/get_product/rfi_g...,NaN,https://s.rfi.fr/media/display/803ddb90-5fd0-1...,http://www.rfi.fr/fr/podcasts/sessionlab/,RFI,SessionLab


### Saved rss file in a folder

In [3]:
def download_rss(url, output_folder, tech_id):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            content = response.content
            output_filename = os.path.join(output_folder, f"{tech_id}.xml")
            with open(output_filename, 'ab') as f:
                f.write(content)
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")

def download_rss_async(df, output_folder, max_workers):
    urls_and_ids = df[['UGC_TECH_RSS_URL', 'UGC_TECH_ID']].to_records(index=False)
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for url, tech_id in urls_and_ids:
            futures.append(executor.submit(download_rss, url, output_folder, tech_id))
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            pass

output_folder = 'rss'
max_workers = 20

download_rss_async(df, output_folder, max_workers)


 12%|█▏        | 610/5298 [00:26<02:04, 37.67it/s]

Error downloading http://podcastpresence.streamakaci.com/xml/clefspourvivre.xml: HTTPConnectionPool(host='podcastpresence.streamakaci.com', port=80): Max retries exceeded with url: /xml/clefspourvivre.xml (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc08d6ce890>: Failed to establish a new connection: [Errno -2] Name or service not known'))
Error downloading http://podcast13.streamakaci.com/xml/GAMEONE26.xml: HTTPConnectionPool(host='podcast13.streamakaci.com', port=80): Max retries exceeded with url: /xml/GAMEONE26.xml (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc08e6970d0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 13%|█▎        | 692/5298 [00:31<03:38, 21.05it/s]

Error downloading http://rss.allocine.fr/bandesannonces/ipod: HTTPConnectionPool(host='rss.allocine.fr', port=80): Max retries exceeded with url: /bandesannonces/ipod (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc08d7ac8e0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 15%|█▌        | 795/5298 [00:36<02:52, 26.18it/s]

...

Error downloading https://cristaletmandarine.com/feed/podcast/: HTTPSConnectionPool(host='cristaletmandarine.com', port=443): Max retries exceeded with url: /feed/podcast/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc08c32bc10>: Failed to establish a new connection: [Errno -2] Name or service not known'))


 92%|█████████▏| 4868/5298 [07:58<00:39, 11.01it/s]

Error downloading https://soufiane-abou-ayyoub.com/feed/podcast: HTTPSConnectionPool(host='soufiane-abou-ayyoub.com', port=443): Max retries exceeded with url: /feed/podcast (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc08c30a5c0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


100%|██████████| 5298/5298 [17:22<00:00,  5.08it/s]

Error downloading https://podcast.agnosys.fr/feed/podcast/ua: HTTPSConnectionPool(host='podcast.agnosys.fr', port=443): Read timed out. (read timeout=None)


### Create a dataset of audio link to podcast

In [43]:
import feedparser

preprocessed_ds = []


for row in tqdm(df.iterrows()):
    filename = f"rss/{row[1]['UGC_TECH_ID']}.xml"
    if os.path.exists(filename):
        feed = feedparser.parse(filename)

        for entry in feed.entries:
            if "enclosures" in entry and len(entry.enclosures) > 0:
                for enclosure in entry.enclosures:
                    if "type" in enclosure and enclosure.type == "audio/mpeg":
                        title = entry.title
                        link = enclosure.href
                        publication_date = entry.published if "published" in entry else "nan"

                        duration = "nan"
                        if hasattr(entry, 'itunes_duration'):
                            duration = entry.itunes_duration
                        elif hasattr(entry, 'itunes_duration'):
                            duration = entry.itunes_duration[0]

                        preprocessed_ds.append([row[1]['UGC_TECH_ID'], row[1]['UGC_TECH_DATE_ENTREE_COLLECTE'], row[1]['UGC_TECH_RSS_URL'], title, publication_date, duration, link])

5298it [08:34, 10.29it/s] 


In [44]:
len(preprocessed_ds)

281759

In [45]:
column_names = ['programme_id', 'programme_entry_date', 'programme_rss_link', 'podcast_title', 'podcast_date', 'podcast_duration', 'audio_podcast_link']
preprocessed_df = pd.DataFrame(preprocessed_ds, columns=column_names)
preprocessed_df.to_csv("french_podcast.csv", escapechar="§", index=False)